In [ ]:
from mainnet_launch.data_fetching.quotes.top_level_check_exit_liqudity import main

quote_df = await main()

quote_df

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1256dd300>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x116b61060>, 1634908.212543541), (<aiohttp.client_proto.ResponseHandler object at 0x1164de560>, 1634910.665877791)])']
connector: <aiohttp.connector.TCPConnector object at 0x1256dcf10>


NameError: name 'quote' is not defined